In [1]:
import datasets 
from nmt_clean.config_m2m_mul_en import config
from nmt_clean.read_files import dataset_from_folders_m2m
from nmt_clean.preprocess import Many2ManyProcessor

from transformers import AutoTokenizer

Evaluating every 20 training steps.


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")

preprocess = Many2ManyProcessor(tokenizer = tokenizer)



100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 27.24it/s]


  0%|          | 0/124 [00:00<?, ?ba/s]

/home/ali/.pyenv/versions/pytorch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 123780
})

In [5]:
paired_datasets = preprocess.dataset_from_folders_m2m(config["training_subset_paths"])

paired_datasets = datasets.concatenate_datasets(paired_datasets)
paired_datasets = paired_datasets.shuffle(seed=42)
paired_datasets = paired_datasets.flatten_indices()  # rewrite the shuffled dataset on disk as contiguous chunks of data

preprocessing_function = preprocess.attach_tokenizer(tokenizer)

training_dataset = paired_datasets.map(preprocessing_function, remove_columns=["translation"], batched=True)

#training_tokens = preprocess.iterative_preprocess(paired_datasets,tokenizer )


#tokens = preprocess.iterative_preprocess(paired_datasets,tokenizer )

100%|█████████████████████████████████████████████| 6/6 [00:07<00:00,  1.20s/it]


Flattening the indices:   0%|          | 0/3684 [00:00<?, ?ba/s]

  0%|          | 0/3684 [00:00<?, ?ba/s]

empty string
empty string
empty string
empty string
empty string


In [9]:
training_dataset[0]

{'input_ids': [62,
  233,
  406,
  246,
  14,
  67,
  113,
  14,
  701,
  6,
  539,
  907,
  44,
  184,
  469,
  11,
  111,
  61,
  15,
  6,
  188,
  256,
  12,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [2110,
  26969,
  37,
  300,
  79,
  186,
  97,
  475,
  2553,
  11909,
  721,
  245,
  694,
  28,
  2943,
  47,
  579,
  245,
  32,
  213,
  292,
  245,
  150,
  45724,
  2337,
  12,
  0]}

In [14]:
val_paired_datasets = preprocess.dataset_from_folders_m2m(config["validation_subset_paths"])

val_paired_datasets = datasets.concatenate_datasets(val_paired_datasets)
#val_paired_datasets = paired_datasets.shuffle(seed=42)
#val_paired_datasets = paired_datasets.flatten_indices()  # rewrite the shuffled dataset on disk as contiguous chunks of data


valid_dataset = val_paired_datasets.map(preprocessing_function, remove_columns=["translation"], batched=True)

#training_tokens = preprocess.iterative_preprocess(paired_datasets,tokenizer )


#tokens = preprocess.iterative_preprocess(paired_datasets,tokenizer )

100%|███████████████████████████████████████████████| 6/6 [00:00<00:00, 21.76it/s]


  0%|          | 0/124 [00:00<?, ?ba/s]

In [17]:
training_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3683940
})

In [16]:
valid_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 123780
})

In [20]:
test_data_dict =  {
            "ach":{
                "all":[config['data_dir'] + "v7.0/supervised/mul-en/test_ach.src"],
                "en":[],
                "lgg":[],
                "lug":[],
                "nyn":[],
                "teo":[]
            },
            "en": {
                "all":[config['data_dir'] + "v7.0/supervised/mul-en/test_ach.tgt"], #all tgts here are the same file 
                "ach":[],
                "lgg":[],
                "lug":[],
                "nyn":[],
                "teo":[]
            },
            "lgg":{
                "all":[config['data_dir'] + "v7.0/supervised/mul-en/test_lgg.src"],
                "en":[],
                "ach":[],
                "lug":[],
                "nyn":[],
                "teo":[]
            },
            "lug":{
                "all":[config['data_dir'] + "v7.0/supervised/mul-en/test_lug.src"],
                "en":[],
                "lgg":[],
                "ach":[],
                "nyn":[],
                "teo":[]
            },
            "nyn":{
                "all":[config['data_dir'] + "v7.0/supervised/mul-en/test_nyn.src"],
                "en":[],
                "lgg":[],
                "lug":[],
                "ach":[],
                "teo":[]
            },
            "teo":{
                "all":[config['data_dir'] + "v7.0/supervised/mul-en/test_teo.src"],
                "en":[],
                "lgg":[],
                "lug":[],
                "nyn":[],
                "ach":[]
            }
}


In [21]:
test_paired_datasets = preprocess.dataset_from_folders_m2m(test_data_dict)

test_paired_datasets = datasets.concatenate_datasets(test_paired_datasets)
#val_paired_datasets = paired_datasets.shuffle(seed=42)
#val_paired_datasets = paired_datasets.flatten_indices()  # rewrite the shuffled dataset on disk as contiguous chunks of data


test_dataset = test_paired_datasets.map(preprocessing_function, remove_columns=["translation"], batched=True)

#training_tokens = preprocess.iterative_preprocess(paired_datasets,tokenizer )


#tokens = preprocess.iterative_preprocess(paired_datasets,tokenizer )

100%|███████████████████████████████████████████████| 6/6 [00:00<00:00, 20.07it/s]


  0%|          | 0/124 [00:00<?, ?ba/s]

In [24]:
whole_dataset = datasets.DatasetDict({
    "train": training_dataset, 
    "validation": valid_dataset,
    "test":test_dataset
})

In [26]:
whole_dataset.push_to_hub("mekaneeky/salt_m2m_ready", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1228 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1228 [00:00<?, ?ba/s]

...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/124 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def _file_to_list(path):
    with open(path) as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
        return lines
    
def dataset_from_src_tgt_files(data_dir, dataset_id, read_first_n = 0):
    path = os.path.join(data_dir, dataset_id)
    source, target = [_file_to_list(path + '.src'),
                      _file_to_list(path + '.tgt')]
    if read_first_n:
        source = source[:read_first_n]
        target = target[:read_first_n]
    pairs = {'translation': [{config['source_language']: s,
                              config['target_language']: t}
                             for s, t in zip(source, target)]}
    return datasets.Dataset.from_dict(pairs)


In [10]:
def _generate_pair_dataset(cls, src_path,src_language, tgt_path, tgt_language, validation_cutoff =0, mode="cutoff_minimum", marian_style_tokens = True):

        src_scentences = cls.load_files(src_path)
        tgt_scentences = cls.load_files(tgt_path)


        if validation_cutoff:
            if mode == "cutoff_maximum":
                src_scentences = src_scentences[:validation_cutoff]
                tgt_scentences = tgt_scentences[:validation_cutoff]
            elif mode == "cutoff_minimum":
                src_scentences = src_scentences[validation_cutoff:]
                tgt_scentences = tgt_scentences[validation_cutoff:]

        #TODO add tokens from tokenizer
        if marian_style_tokens:
            src_scentences = [tgt_language + " " + src for src in src_scentences]
            #tgt_scentences = [language_token_dict[tgt_language] + " " + tgt for tgt in tgt_scentences]


        pairs = {'translation': [{"src": s,
                            "tgt": t}
                            for s, t in zip(src_scentences, tgt_scentences)]
        }
        paired_dataset = datasets.Dataset.from_dict(pairs)
        paired_dataset.src_language = src_language
        paired_dataset.tgt_language = tgt_language
        
        return paired_dataset

In [2]:
total_count = 0
for i in range(len(tokens)):
    total_count += len(test[i])
    #print(f"Src:{test[i].src_language} Tgt:{test[i].tgt_language} Count:{len(test[i])}")
    
print(total_count)

NameError: name 'tokens' is not defined

['val_teo.src',
 'test_ach.src',
 'train_mt560_luo.src',
 'val_nyn.src',
 'val_teo.tgt',
 'train_mt560_ach.tgt',
 'val_lug.src',
 'val_nyn.tgt',
 'test_lgg.tgt',
 'test_nyn.tgt',
 'train_flores_lug.tgt',
 'train_ai4d.tgt',
 'val_lgg.src',
 'test_lug.tgt',
 'train.src',
 'train_mt560_ach.src',
 'val_ach.tgt',
 'bukedde_ggl_bt_lug.tgt',
 'test_teo.src',
 'test_teo.tgt',
 'train.tgt',
 'train_ai4d.src',
 'train_flores_luo.src',
 'val_lgg.tgt',
 'test_nyn.src',
 'train_mt560_nyn.tgt',
 'val_lug.tgt',
 'train_mt560_luo.tgt',
 'test_ach.tgt',
 'test_lgg.src',
 'train_mt560_lug.src',
 'train_flores_lug.src',
 'val_ach.src',
 'test_lug.src',
 'train_flores_luo.tgt',
 'train_mt560_nyn.src',
 'train_mt560_lug.tgt',
 'bukedde_ggl_bt_lug.src']